In [492]:
import random

In [496]:
stake_return = {
    "lose": lambda stake: 0,
    "push": lambda stake: stake,
    "blackjack": lambda stake: stake + int(stake * 1.5),
    "win": lambda stake: stake + stake
}

stake_return['blackjack'](10)

25

In [3]:
from itertools import product

player_state = ["P_BJ", "P_BUST", "P_SCORE"]
dealer_state = ["D_BJ", "D_BUST", "D_SCORE"]

list(product(player_state, dealer_state))



[('P_BJ', 'D_BJ'),
 ('P_BJ', 'D_BUST'),
 ('P_BJ', 'D_SCORE'),
 ('P_BUST', 'D_BJ'),
 ('P_BUST', 'D_BUST'),
 ('P_BUST', 'D_SCORE'),
 ('P_SCORE', 'D_BJ'),
 ('P_SCORE', 'D_BUST'),
 ('P_SCORE', 'D_SCORE')]

In [493]:
class Card():
    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit

    def __repr__(self):
        return f"{self.rank} of {self.suit}"

    def show(self):
        print(f"{self.rank} of {self.suit}")


class Deck():
    def __init__(self):
        self.cards = []
        self.new_deck()
        self.shuffle()

    def new_deck(self):
        suits = ['♥', '♣', '♦', '♠']
        ranks = ['A', '2', '3', '4', '5', '6',
                 '7', '8', '9', '10', 'J', 'Q', 'K']
        self.cards = [Card(rank, suit) for rank in ranks for suit in suits]

    def shuffle(self):
        random.shuffle(self.cards)

    def get_card(self):
        return self.cards.pop()


class Player():
    def __init__(self, name, chips=100):
        self.name = name
        self.chips = chips

    def __repr__(self):
        return self.name


class Blackjack():
    card_values = {'A': 11, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6,
                   '7': 7, '8': 8, '9': 9, '10': 10, 'J': 10, 'Q': 10, 'K': 10}
    stake_return = {
        "loss": lambda stake: 0,
        "push": lambda stake: stake,
        "blackjack": lambda stake: stake + int(stake * 1.5),
        "win": lambda stake: stake + stake
    }

    def __init__(self):
        self.deck = Deck()
        self.players = []
        self.players_hands = {}
        self.players_bets = {}

    def play(self):
        def get_hand_score(hand):
            score = 0
            aces = 0
            # get_score
            for card in hand:
                if card.rank == 'A':
                    aces += 1
                score += self.card_values[card.rank]
                while score > 21 and aces:
                    aces -= 1
                    score -= 10
            return score

        def get_hand_result(player_hand, dealer_hand):
            dealer_bust = False
            dealer_blackjack = False
            player_bust = False
            player_blackjack = False

            dealer_score = get_hand_score(dealer_hand)
            player_score = get_hand_score(player_hand)

            if dealer_score == 21 and len(dealer_hand) == 2:
                dealer_blackjack = True

            if dealer_score > 21:
                dealer_bust = True

            if player_score == 21 and len(hand) == 2:
                player_blackjack = True

            if player_score > 21:
                player_bust = True

            if player_bust:
                if dealer_bust:
                    return self.stake_return['push']
                else:
                    return self.stake_return['loss']
            if player_blackjack:
                if dealer_blackjack:
                    return self.stake_return['push']
                else:
                    return self.stake_return['win']
            if player_score == dealer_score:
                return self.stake_return['push']
            if player_score > dealer_score:
                return self.stake_return['win']
            if player_score < dealer_score:
                return self.stake_return['loss']

            return None

        # if input("All players added?:").upper().startswith("N"):
            # return None
        if not self.players:
            return None
        # self.setup()
        print("-- Remove Poor Players --")
        for player in self.players:
            print(player, player.chips)
            if player.chips >= 10:
                self.players_bets[player] = 10
                print(player, "kept. Can cover bet.")
            else:
                self.players.remove(player)
                print(player, "ejected. Cannot cover bet.")

        print("-- Take Bet --")
        for player in self.players:
            print("Before:", player, player.chips)
            player.chips -= 10
            print("After:", player, player.chips)

        for player in self.players:
            self.players_hands[player] = []

        # self.deal()
        for player in self.players*2:
            self.players_hands[player].append(self.deck.get_card())

        dealer_hand = []
        dealer_hand.append(self.deck.get_card())
        dealer_hand.append(self.deck.get_card())
        print("Opening Hands:")
        print('\t', "Dealer", [dealer_hand[0]])
        for name, hand in self.players_hands.items():
            print('\t', name, hand)

        # main loop, for each player
        # Hit or Stick until Stick or bust.
        for name, hand in self.players_hands.items():
            # for _ in range(5):
            #     hand.append(Card('A', '♠'))
            while True:
                # show hand, score then hit or Stick
                score = get_hand_score(hand)
                print(name, score, hand)
                # check_bust
                if score > 21:
                    # print("Final score:", score)
                    break
                # hit_or_stick
                choice = input("[H]it or [S]tick?: ")
                if choice == "S":
                    # print("Final score:", score)
                    break
                else:
                    hand.append(self.deck.get_card())
            # Player has no more decisions. Stuck, or Bust
        # All Players have made their decisions
        # Play through the Dealer's turn
        dealer_score = get_hand_score(dealer_hand)
        print("Dealer", dealer_score, dealer_hand)
        while dealer_score < 17:
            dealer_hand.append(self.deck.get_card())
            dealer_score = get_hand_score(dealer_hand)
            print("Dealer", dealer_score, dealer_hand)

        # Get the scores any payout winners
        print("-- Resolving Bets --")
        print(self.players_bets)
        for name, hand in self.players_hands.items():
            hand_score = get_hand_score(hand)
            player_bet = self.players_bets[name]
            bet_result = ""
            # print("Bet:", self.players_bets[name])

            dealer_bust = False
            dealer_blackjack = False
            hand_bust = False
            hand_blackjack = False

            if dealer_score == 21 and len(dealer_hand) == 2:
                dealer_blackjack = True

            if dealer_score > 21:
                dealer_bust = True

            if hand_score == 21 and len(hand) == 2:
                hand_blackjack = True
            if hand_score > 21:
                hand_bust = True

            # dealer blackjack
            if dealer_blackjack:
                # dealer blackjack
                if hand_blackjack:
                    bet_result = "push"
                else:
                    bet_result = "loss"
            # dealer bust
            elif dealer_bust:
                if hand_blackjack:
                    bet_result = "push"
                else:
                    bet_result = "win"
            # score vs score
            else:
                if hand_score == dealer_score:
                    bet_result = "push"
                elif hand_score > dealer_score:
                    bet_result = "win"
                else:
                    bet_result = "win"

            # check if blackjack
            result = get_hand_result(player_hand=hand, dealer_hand=dealer_hand)

            bet_return = result(player_bet)
            name.chips += bet_return

            print(name, bet_result, bet_return, name.chips)


In [494]:
p1 = Player("Cameron", chips=25)
p2 = Player("Hayley")

game = Blackjack()
game.players.append(p1)
game.players.append(p2)
game.players

[Cameron, Hayley]

In [497]:
game.play()
# print(game.players_hands)

-- Remove Poor Players --
Cameron 35
Cameron kept. Can cover bet.
Hayley 110
Hayley kept. Can cover bet.
-- Take Bet --
Before: Cameron 35
After: Cameron 25
Before: Hayley 110
After: Hayley 100
Opening Hands:
	 Dealer [A of ♥]
	 Cameron [6 of ♠, K of ♦]
	 Hayley [J of ♦, 5 of ♥]
Cameron 16 [6 of ♠, K of ♦]
Hayley 15 [J of ♦, 5 of ♥]
Dealer 21 [A of ♥, Q of ♦]
-- Resolving Bets --
{Cameron: 10, Hayley: 10}
Cameron loss 0 25
Hayley loss 0 100


### Potential Changes
- Dealer Stick - Soft vs Hard Ace
- 5 card special win?
- Insurance
- Splititng
- Surrender
- Doubling Down